## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [160]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
# df.head()

In [161]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [162]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [163]:
# Use this space to do any additional research
# and familiarize yourself with the data.
print(f"The shape of the frame is {df.shape}\n")

The shape of the frame is (54639, 18)



In [164]:
# What three item categories had the most entries?
top_3 = df["category"].value_counts()[:3]
# top_3

In [165]:
# For the category with the most entries,
# which subcategory had the most entries?
df.loc[df["category"] == 'consumables']["subcategory"].value_counts()[:1]

subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [166]:
# Which five clients had the most entries in the data?
df["client_id"].value_counts()[:5]

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [167]:
# Store the client ids of those top 5 clients in a list.
top_5_clients = list(df["client_id"].value_counts().keys()[:5])
# top_5_clients

In [168]:
# How many total units (the qty column) did the
# client with the most entries order order?
tot_qty = df.loc[df["client_id"] == top_5_clients[0]]["qty"].sum()
print(f"Client {top_5_clients[0]} ordered a total quantity of {tot_qty} items.")

Client 33615 ordered a total quantity of 64313 items.


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [169]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["subtotal"] = df["qty"]*df["unit_price"]
# df.head()

In [170]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def shipping_price(row):
    order_weight = row["unit_weight"]*row["qty"]
    if order_weight > 50:
        price = 7*order_weight
        return price
    price = order_weight * 10
    return price

df["shipping_price"]=df.apply(shipping_price, axis=1)
# df.head()


In [171]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

def total_price(row):
    tot_cost = round((row["subtotal"]+row["shipping_price"])*1.0925,2)
    return tot_cost

df["line_price"] = df.apply(total_price,axis=1)

# df[["order_id","subtotal","shipping_price","line_price"]]

In [172]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

def cost_calc(row):
    cost = (row["unit_cost"]*row["qty"])+row["shipping_price"]
    return cost

df["line_cost"] = df.apply(cost_calc,axis=1)
# df.head()

In [173]:
# Create a column for the profit of
# each line using line cost and line price
def line_profit(row):
    tot_profit = row["line_price"]-row["line_cost"]
    return tot_profit

df["line_profit"] = df.apply(line_profit,axis=1)

# df.head()


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [174]:
# Check your work using the totals above
check1 = round(df.loc[df["order_id"] == 2742071]["line_price"].sum(),2)
print(f"Order ID 2742071 had a total price of {check1}")

check2 = round(df.loc[df["order_id"] == 2173913]["line_price"].sum(),2)
print(f"Order ID 2742071 had a total price of {check2}")

check3 = round(df.loc[df["order_id"] == 6128929]["line_price"].sum(),2)
print(f"Order ID 2742071 had a total price of {check3}")

Order ID 2742071 had a total price of 152811.89
Order ID 2742071 had a total price of 162388.71
Order ID 2742071 had a total price of 923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [175]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
for client in top_5_clients:
    client_total = round(df.loc[df["client_id"] == client]["line_price"].sum(),2)
    print(f"Client {client} spent ${client_total}.")


Client 33615 spent $8377308.53.
Client 66037 spent $10259514.8.
Client 46820 spent $9743794.33.
Client 38378 spent $12906550.87.
Client 24741 spent $82268892.04.


In [188]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
summary_data = []
for client in top_5_clients:
    client_qty = df.loc[df["client_id"] == client]["qty"].sum()
    client_ship = round(df.loc[df["client_id"] == client]["shipping_price"].sum(),2)
    client_revenue = round(df.loc[df["client_id"] == client]["line_price"].sum(),2)
    client_cost = round(df.loc[df["client_id"] == client]["line_cost"].sum(),2)
    client_profit = round(df.loc[df["client_id"] == client]["line_profit"].sum(),2)
    client_data = {"client_id":client,
                   "qty":client_qty,
                   "total_shipping":client_ship,
                   "total_revenue":client_revenue,
                   "total_cost":client_cost,
                   "total_profit":client_profit}
    summary_data.append(client_data)

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values("total_profit", ascending=False)


In [189]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.
summary_df = summary_df.rename(columns={"client_id":"Client",
                           "qty":"Quantity",
                           "total_shipping":"Total Shipping ($M)",
                           "total_revenue":"Total Revenue ($M)",
                           "total_cost":"Total Cost ($M)",
                           "total_profit":"Total Profit ($M)"})

columns_to_scale = ["Total Shipping ($M)","Total Revenue ($M)","Total Cost ($M)","Total Profit ($M)"]

#if statement just prevents repeatedly scaling data if cell is run multiple times
if summary_df["Total Cost ($M)"].sum() > 10000:
    for column in columns_to_scale:   
        summary_df[column] = round(summary_df[column]/1000000,2)

summary_df

,Client,Quantity,Total Shipping ($M),Total Revenue ($M),Total Cost ($M),Total Profit ($M)
4,24741,239862,5.13,82.27,45.69,36.58
3,38378,73667,3.43,12.91,9.63,3.27
1,66037,43018,1.40,10.26,7.00,3.26
2,46820,75768,1.60,9.74,7.01,2.74
0,33615,64313,1.83,8.38,6.18,2.20


In [190]:
# Sort the updated data by "Total Profit" form highest to lowest

# Sorted at inception above and saved
summary_df

,Client,Quantity,Total Shipping ($M),Total Revenue ($M),Total Cost ($M),Total Profit ($M)
4,24741,239862,5.13,82.27,45.69,36.58
3,38378,73667,3.43,12.91,9.63,3.27
1,66037,43018,1.40,10.26,7.00,3.26
2,46820,75768,1.60,9.74,7.01,2.74
0,33615,64313,1.83,8.38,6.18,2.20
